In [385]:
import collections
import datetime
import requests
import inflection
import pandas
import pytz

In [386]:
pandas.set_option('max_rows', None)
pandas.set_option('max_columns', None)

In [387]:
STALE_DT = (datetime.datetime.utcnow() - datetime.timedelta(minutes=90)).replace(tzinfo=pytz.utc)

In [388]:
STALE_DT

datetime.datetime(2016, 11, 8, 22, 41, 20, 535801, tzinfo=<UTC>)

In [389]:
strikes_df['call__updated'].max() - STALE_DT

Timedelta('-1 days +23:09:22.464199')

In [390]:
def strip_updated(updated_at):
    updated_at = updated_at.replace('CT<br /> ', '')
    dt = datetime.datetime.strptime(updated_at, '%H:%M:%S %d %b %Y')
    dt = dt.replace(tzinfo=pytz.timezone('US/Central'))
    dt = dt.astimezone(pytz.utc)
    return dt

In [391]:
# url = 'http://www.cmegroup.com/CmeWS/mvc/Quotes/Option/2916/G/X6/Active?optionProductId=2915&strikeRange=Active&pageSize=50&_=1478643918531'
url = 'https://query1.finance.yahoo.com/v7/finance/options/SPY?formatted=true&crumb=oPWCX4Hh1ne&lang=en-US&region=US&straddle=false&corsDomain=finance.yahoo.com'

In [392]:
req = requests.get(url)

In [393]:
data = req.json()

In [394]:
# line_cols = ['strikePrice', 'strikeRank', 'underlyingFutureContract']

# strikes_list = []
# for strike_line in data['optionContractQuotes']:
#     to_append = {inflection.underscore(c): strike_line[c] for c in line_cols}
    
#     to_append.update({'put__{c}'.format(c=c): v for c, v in strike_line['put'].items()})
#     to_append.update({'call__{c}'.format(c=c): v for c, v in strike_line['call'].items()})
    
#     to_append['put__updated'] = strip_updated(to_append['put__updated'])
#     to_append['call__updated'] = strip_updated(to_append['call__updated'])
    
#     strikes_list.append(to_append)
# strikes_df = pandas.DataFrame(strikes_list)

In [395]:
# strikes_df['strike_price'] = strikes_df['strike_price'].astype(float)
# strikes_df['call__last'] = strikes_df['call__last'].apply(lambda k: float(k) if k != '-' else None)
# strikes_df['put__last'] = strikes_df['put__last'].apply(lambda k: float(k) if k != '-' else None)

In [396]:
# non_stale_slice = strikes_df['call__updated'] > STALE_DT
# mod_1000_strike = strikes_df['strike_price'] % 1000 == 0
# within_fwd      = strikes_df['strike_price'].apply(lambda k: abs(k-214000) < 8000)

In [397]:
# calls = strikes_df[non_stale_slice & mod_1000_strike & within_fwd][['strike_price','call__last','call__updated']]
# calls['call__spread'] = calls['call__last'].diff(-1)
# calls['call__fly'] = calls['call__spread'].diff(-1)
# calls['fly_imp_prob'] = calls['call__fly'] / 10

In [398]:
opts = data['optionChain']['result'][0]['options'][0]

In [399]:
MIN_STRIKE = int(min(min(opt['strike']['raw'] for opt in opts['calls']), min(opt['strike']['raw'] for opt in opts['puts'])))
MAX_STRIKE = int(max(max(opt['strike']['raw'] for opt in opts['calls']), max(opt['strike']['raw'] for opt in opts['puts'])))

In [400]:
MIN_STRIKE, MAX_STRIKE

(175, 235)

In [401]:
calls = {}
puts  = {}

for call_dict in opts['calls']:
    calls[call_dict['strike']['raw']] = call_dict
for puts_dict in opts['puts']:
    puts[puts_dict['strike']['raw']] = puts_dict

In [402]:
options_list = []
for k in range(MIN_STRIKE, MAX_STRIKE+1, 1):
    call_dict = calls.get(k, None)
    put_dict = puts.get(k, None)
    
    to_append = {}
    
    if call_dict:
        to_append.update({
            'call_strike': call_dict['strike']['raw'],
            'call_expiration': datetime.datetime.fromtimestamp(call_dict['expiration']['raw']),
            'call_ask': call_dict['ask']['raw'],
            'call_bid': call_dict['bid']['raw'],
            'call_mm': (call_dict['ask']['raw'] + call_dict['bid']['raw']) / 2,
            'call_symbol': call_dict['contractSymbol'],
            'call_open_interest':  call_dict['openInterest']['raw'],
        })
    
    if put_dict:
        to_append.update({
            'put_strike': put_dict['strike']['raw'],
            'put_expiration': datetime.datetime.fromtimestamp(put_dict['expiration']['raw']),
            'put_ask': put_dict['ask']['raw'],
            'put_bid': put_dict['bid']['raw'],
            'put_mm': (put_dict['ask']['raw'] + put_dict['bid']['raw']) / 2,
            'put_symbol': put_dict['contractSymbol'],
            'put_open_interest': put_dict['openInterest']['raw'],
        })
    options_list.append(to_append)
df = pandas.DataFrame(options_list)

In [403]:
MIN_STRIKE = 205
MAX_STRIKE = 223
STRIKE_LIST = list(range(MIN_STRIKE, MAX_STRIKE+1, 1))

In [404]:
df = df[df['call_strike'].isin(STRIKE_LIST)].reset_index(drop=True)

In [405]:
df['call_spread'] = df['call_mm'] - df['call_mm'].shift(-1)
df['call_fly']    = df['call_spread'] - df['call_spread'].shift(-1)

df['put_spread'] = df['put_mm'] - df['put_mm'].shift(-1)
df['put_fly']    = df['put_spread'] - df['put_spread'].shift(-1)

In [417]:
df['fly'] = df.apply(lambda k: k['call_fly'] if k['call_strike'] > 215 else k['put_fly'], axis=1).round(decimals=4)

In [418]:
df[['fly','call_strike','call_fly','call_mm','put_strike','put_fly','put_mm']]

,fly,call_strike,call_fly,call_mm,put_strike,put_fly,put_mm
0,0.065,205.0,-0.020,9.585,205.0,6.500000e-02,0.540
1,0.000,206.0,0.040,8.695,206.0,0.000000e+00,0.625
2,-0.025,207.0,0.010,7.785,207.0,-2.500000e-02,0.775
3,-0.000,208.0,-0.040,6.915,208.0,-2.220446e-16,0.925
4,0.050,209.0,0.125,6.055,209.0,5.000000e-02,1.050
5,0.055,210.0,-0.025,5.155,210.0,5.500000e-02,1.175
6,-0.020,211.0,0.010,4.380,211.0,-2.000000e-02,1.350
7,0.020,212.0,0.075,3.580,212.0,2.000000e-02,1.580
8,0.135,213.0,0.035,2.790,213.0,1.350000e-01,1.790
9,0.065,214.0,0.095,2.075,214.0,6.500000e-02,2.020
